In [1]:
"""
Extract TreeMap (c.a. 2016) statistics in active fire detections
    - calculate the count of pixels for each forest species in gridcells (percent cover)
    - for each species (masked), calculate the mean TreeMap metrics (BALIVE, SDI, STANDHT)
    - for each species (masked), calculate the mean Sentinel-2 metrics (LAI, MNDWI)
Author: maxwell.cook@colorado.edu
"""

import ee, geemap
import os, sys, time
import pandas as pd

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Success")

Success


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [ ]:
# load and prep the USFS TreeMap

In [2]:
treemap = ee.ImageCollection("USFS/GTAC/TreeMap/v2016")
print(f"TreeMap bands available for analysis:\n\n{treemap.first().bandNames().getInfo()}")

TreeMap bands available for analysis:

['ALSTK', 'BALIVE', 'CANOPYPCT', 'CARBON_D', 'CARBON_DWN', 'CARBON_L', 'DRYBIO_D', 'DRYBIO_L', 'FLDSZCD', 'FLDTYPCD', 'FORTYPCD', 'GSSTK', 'QMD_RMRS', 'SDIPCT_RMRS', 'STANDHT', 'STDSZCD', 'TPA_DEAD', 'TPA_LIVE', 'Value', 'VOLBFNET_L', 'VOLCFNET_D', 'VOLCFNET_L']


In [3]:
# grab the metrics we care about
treemap = treemap.select(['FORTYPCD','BALIVE','SDIPCT_RMRS','STANDHT','TPA_LIVE','TPA_DEAD'])
treemap.first().bandNames().getInfo()

['FORTYPCD', 'BALIVE', 'SDIPCT_RMRS', 'STANDHT', 'TPA_LIVE', 'TPA_DEAD']

In [4]:
treemap

In [5]:
# extract the class code table, create a dictionary
class_codes = treemap.first().get('FORTYPCD_class_values').getInfo()
class_names = treemap.first().get('FORTYPCD_class_names').getInfo()
code_to_name = dict(zip(class_codes, class_names)) # link class code to name

# convert to a data frame and export
species_df = pd.DataFrame(list(code_to_name.items()), columns=['FORTYPCD', 'SpeciesName'])
species_df['FORTYPCD'] = species_df['FORTYPCD'].astype(int)
species_df = species_df.reset_index(drop=True)
species_df.head()

,FORTYPCD,SpeciesName
0,101,Jack pine
1,102,Red pine
2,103,Eastern white pine
3,104,Eastern white pine / eastern hemlock
4,105,Eastern hemlock


In [6]:
print(species_df[species_df['SpeciesName'] == 'Ponderosa pine']) # check accuracy
print(species_df[species_df['SpeciesName'] == 'Lodgepole pine']) 

    FORTYPCD     SpeciesName
27       221  Ponderosa pine
    FORTYPCD     SpeciesName
44       281  Lodgepole pine


In [7]:
# Save this file out.
out_fp = os.path.join(projdir,'data/tabular/mod/treemap_fortypcd_species_mapping.csv')
species_df.to_csv(out_fp)
print(f"Dictionary saved to {out_fp}")

Dictionary saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/tabular/mod/treemap_fortypcd_species_mapping.csv


In [ ]:
# load the gridded FRP data for aspen fires

In [9]:
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar_gridstats')
print(f"{grid.size().getInfo()} total gridcells.")
print(grid.first().propertyNames().getInfo())

49047 total gridcells.
['Fire_Year', 'grid_index', 'Fire_ID', 'max_date', 'afd_count', 'Ig_Date', 'first_date', 'last_date', 'Last_Dat_1', 'system:index']


In [10]:
grid.limit(10)

In [11]:
# calculate the species histogram

In [11]:
constant = ee.Image.constant(1) # creates a constant image
constant = constant.reproject(treemap.mosaic().projection()) # scale to treemap projection

In [12]:
def species_histogram(ftr):
    """ Generates a histogram of occurrence in a region """
    image = treemap.select('FORTYPCD').mosaic()
    
    # calculate the histogram
    sp_hist = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=ftr.geometry(),
        scale=30
    ).get('FORTYPCD')

    # grab the total pixels used in calculation
    total_pixels = constant.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=ftr.geometry(),
        scale=30
    ).get('constant')

    sp_hist_json = ee.Dictionary(sp_hist).map(
        lambda key, value: ee.String(key).cat(':').cat(ee.Number(value).format())
    ).values().join(', ')

    # return the histogram dictionary without unpacking
    return ftr.set({
        'species_histogram': sp_hist_json,
        'total_pixels': total_pixels
    })

# map across gridcells
grid = grid.select(['grid_index','.geo']) # just keep the grid ID and geometry
fortypcd = grid.map(species_histogram) # apply the function to the grid
print("Process submitted to the server !")

Process submitted to the server !


In [ ]:
# check the results

In [13]:
fortypcd.first().propertyNames().getInfo()

['total_pixels', 'species_histogram', 'system:index', 'grid_index']

In [14]:
sample = fortypcd.limit(10).getInfo()
props = [f['properties'] for f in sample['features']]
df = pd.DataFrame(props)
df.head()

,grid_index,species_histogram,total_pixels
0,919906,"182:3.0941176470588236, 185:7.0, 221:41.768627...",196.949020
1,919907,"182:9.227450980392156, 221:75.02745098039216, ...",197.176471
2,919908,"182:15.611764705882354, 185:10.996078431372549...",196.960784
3,922166,"182:8.36078431372549, 184:16.83137254901961, 1...",197.176471
4,922171,"182:9.768627450980393, 184:2.0, 185:4.05098039...",197.466667


In [15]:
# unpack histogram dictionary into columns
def parse_histogram(hist_str):
    kv_pairs = hist_str.split(', ')
    return {int(kv.split(':')[0]): float(kv.split(':')[1]) for kv in kv_pairs}
df['species_histogram'] = df['species_histogram'].apply(parse_histogram)
df = df['species_histogram'].apply(pd.Series)
print("Adjusted DataFrame:")
df.head(10)

Adjusted DataFrame:


,182,185,221,225,369,371,901,971,922,184,...,706,281,703,261,267,268,999,224,266,974
0,3.094118,7.000000,41.768627,36.423529,2.674510,11.607843,0.494118,4.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9.227451,NaN,75.027451,32.870588,12.109804,6.827451,NaN,1.000000,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15.611765,10.996078,37.329412,15.466667,12.000000,2.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.360784,46.721569,44.247059,9.231373,20.058824,NaN,NaN,12.411765,NaN,16.831373,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9.768627,4.050980,72.717647,34.400000,8.690196,9.603922,1.000000,1.996078,NaN,2.000000,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,12.000000,10.333333,32.839216,7.937255,11.435294,1.701961,1.000000,7.909804,NaN,NaN,...,NaN,2.000000,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3.000000,NaN,24.250980,NaN,NaN,1.984314,119.796078,NaN,NaN,NaN,...,NaN,NaN,NaN,4.372549,1.000000,5.0,9.027451,NaN,NaN,NaN
7,6.156863,NaN,19.678431,2.996078,1.000000,3.552941,51.325490,0.741176,NaN,NaN,...,NaN,4.890196,NaN,2.000000,8.023529,2.0,NaN,NaN,NaN,NaN
8,NaN,NaN,17.682353,3.000000,NaN,27.788235,66.239216,25.023529,NaN,NaN,...,NaN,5.407843,NaN,8.925490,8.588235,NaN,2.000000,1.0,1.0,NaN
9,7.996078,3.000000,58.192157,12.188235,NaN,11.886275,7.886275,9.050980,1.0,NaN,...,NaN,18.007843,NaN,8.168627,4.000000,NaN,NaN,2.0,NaN,1.0


In [16]:
# Export to Asset/Drive
task = ee.batch.Export.table.toDrive(
    collection=fortypcd,
    description='gridstats_fortypcd',
    fileNamePrefix='gridstats_fortypcd',
    fileFormat='CSV',
    folder='TreeMap'
)

task.start() # Start the export task
print("Export to Google Drive started!")
monitor_export(task, timeout=120)

Export to Google Drive started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Export completed successfully !!!!


In [ ]:
# Gather the species metrics

In [17]:
cols = ['BALIVE','SDIPCT_RMRS','STANDHT','TPA_LIVE','TPA_DEAD']
grid = grid.select(['grid_index'])

# function to calculate mean by species
def species_metrics(ftr):
    image = treemap.mosaic()
    
    # Get species histogram
    hist = image.select('FORTYPCD').reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=ftr.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('FORTYPCD')

    hist_dict = ee.Dictionary(hist)

    # Initialize dictionary to store species metrics
    def add_species_metrics(key, current_dict):
        key_str = ee.String(key)
        
        # Mask the image by the species type
        masked_image = image.updateMask(image.select('FORTYPCD').eq(ee.Number.parse(key).toInt()))
        
        # Calculate average metrics for the masked image
        metrics = masked_image.select(cols).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ftr.geometry(),  # Gridcell
            scale=30,
            maxPixels=1e13
        )
        
        # Construct a sub-dictionary for this species
        sp_metrics = ee.Dictionary.fromLists(
            ee.List(cols).map(lambda col: key_str.cat('_').cat(col).cat('_mn')),
            ee.List(cols).map(lambda col: metrics.get(col))
        )
        
        # Combine this species' metrics with the current dictionary
        return ee.Dictionary(current_dict).combine(sp_metrics)

    # Iterate over all species to calculate metrics
    metrics_dict = hist_dict.keys().iterate(add_species_metrics, ee.Dictionary())
    metrics_string = ee.Dictionary(metrics_dict).map(
        lambda key, value: ee.String(key).cat(':').cat(ee.Number(value).format())
    ).values().join(', ')
    
    # Return the feature with the species metrics as a single dictionary property
    return ftr.set({
        'species_metrics': metrics_string,
    })

# map the function over the grids
fortypcd_metrics = grid.map(species_metrics)
print("Submitted !")

Submitted !


In [18]:
sample = fortypcd_metrics.limit(10).getInfo()
props = [f['properties'] for f in sample['features']]
df = pd.DataFrame(props)
df.head()

,grid_index,species_metrics
0,919906,"182_BALIVE_mn:113.51390075683594, 182_SDIPCT_R..."
1,919907,"182_BALIVE_mn:113.51390075683594, 182_SDIPCT_R..."
2,919908,"182_BALIVE_mn:63.25205624256263, 182_SDIPCT_RM..."
3,922166,"182_BALIVE_mn:103.75088572904717, 182_SDIPCT_R..."
4,922171,"182_BALIVE_mn:103.60663347462486, 182_SDIPCT_R..."


In [19]:
# unpack histogram dictionary into columns
def parse_histogram(hist_str):
    kv_pairs = hist_str.split(', ')
    return {kv.split(':')[0]: float(kv.split(':')[1]) for kv in kv_pairs}
df['species_metrics'] = df['species_metrics'].apply(parse_histogram)
df = df['species_metrics'].apply(pd.Series)
print("Adjusted DataFrame:")
df.head(10)

Adjusted DataFrame:


,182_BALIVE_mn,182_SDIPCT_RMRS_mn,182_STANDHT_mn,182_TPA_DEAD_mn,182_TPA_LIVE_mn,185_BALIVE_mn,185_SDIPCT_RMRS_mn,185_STANDHT_mn,185_TPA_DEAD_mn,185_TPA_LIVE_mn,...,224_TPA_LIVE_mn,266_BALIVE_mn,266_STANDHT_mn,266_TPA_DEAD_mn,266_TPA_LIVE_mn,974_BALIVE_mn,974_SDIPCT_RMRS_mn,974_STANDHT_mn,974_TPA_DEAD_mn,974_TPA_LIVE_mn
0,113.513901,65.199997,29.000000,12.036092,396.670441,44.991299,26.083333,20.285714,9.027069,773.867731,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,113.513901,65.199997,29.000000,12.036092,396.670441,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63.252056,33.140844,29.000000,12.036092,146.540987,52.111698,25.000000,30.000000,NaN,428.988831,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,103.750886,57.694653,28.415572,48.841899,473.854566,67.542999,30.973641,24.497902,12.176737,429.010974,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,103.606633,57.993254,29.614211,24.430655,338.867526,83.925314,29.932623,23.493708,12.634444,177.626713,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,65.209068,33.666667,29.500000,60.180462,149.361375,67.694416,31.167742,25.000000,6.018046,491.862930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,32.979301,17.100000,26.000000,12.036092,117.091606,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,66.810949,36.760191,29.025478,6.018046,391.397017,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,383.612244,118.076103,64.0,12.036092,469.070923,NaN,NaN,NaN,NaN,NaN
9,88.638074,51.598332,29.624816,6.018046,540.409072,167.820503,54.333333,26.333333,6.018046,159.217926,...,383.612244,NaN,NaN,NaN,NaN,146.244202,70.599998,28.0,36.108276,522.528809


In [ ]:
# export it.
export_task = ee.batch.Export.table.toDrive(
    collection=fortypcd_metrics,
    description='gridstats_fortypcd_metrics',
    fileNamePrefix='gridstats_fortypcd_metrics',
    fileFormat='CSV', 
    folder='TreeMap'
)

export_task.start() # Start the export task
print("Export to Earth Engine Asset started!")
monitor_export(export_task, 120)